In [ ]:
# Cell 1: Setup
import pandas as pd
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter!
import re
import os
from dotenv import load_dotenv

# Load environment variables from .env file into os.environ
load_dotenv()

# Access them
langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
project_name = os.getenv("LANGCHAIN_PROJECT")
endpoint = os.getenv("LANGCHAIN_ENDPOINT")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")




# # --- IMPORTANT ---
# # Make sure your script with the ResearchAssistant class is saved as a .py file
# # For example, save it as 'sagerag.py' in the same directory.
# from sagerag import ResearchAssistant, OllamaLLM, Chroma, HuggingFaceEmbeddings

# print("🚀 Initializing...")

# # --- Load your NEW, large ChromaDB ---
# DB_DIR_NEW = "../Database/Vector-DB-New" 
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
# vector_db = Chroma(persist_directory=DB_DIR_NEW, embedding_function=embedding_model)

# # --- Initialize the LLM ---
# llm = OllamaLLM(model="llama3.2", temperature=0.2)

# # --- Initialize the main assistant class which we will use as a toolkit ---
# assistant = ResearchAssistant(llm=llm, vector_db=vector_db)

# print("✅ Setup Complete. Assistant is ready.")

In [5]:
# Cell 2: Reusable RAG Pipeline Helper
def run_rag_pipeline(assistant, query_for_retrieval, original_question):
    """
    A non-interactive function to run the retrieve-and-generate pipeline.
    
    Args:
        assistant: The initialized ResearchAssistant object.
        query_for_retrieval: The query to use for finding documents.
        original_question: The original question to be answered by the LLM.
    
    Returns:
        The final generated answer string.
    """
    try:
        # 1. Retrieve documents using the given query
        retrieved_docs, _ = assistant.retrieve_documents(queries=[query_for_retrieval], k=5)
        if not retrieved_docs:
            return "Error: Could not retrieve any relevant documents."
            
        # 2. Generate the final answer, ensuring the LLM answers the *original* question
        final_answer = assistant.generate_final_answer(
            question=original_question,
            docs=retrieved_docs,
            max_docs=5
        )
        return final_answer
    except Exception as e:
        return f"Error in RAG pipeline: {e}"
def run_rag_C_pipeline(assistant, query_for_retrieval, original_question):
    """The MID-TIER RAG pipeline for System C. No HyDE, but uses the CITED prompt."""
    try:
        # Retrieve documents using the query directly
        retrieved_docs, _ = assistant.retrieve_documents(queries=[query_for_retrieval], k=5)
        if not retrieved_docs:
            return "Error: Could not retrieve any relevant documents."
        
        # Call the mid-tier generator with citations
        return assistant.generate_cited_answer(original_question, retrieved_docs)
    except Exception as e:
        return f"Error in RAG pipeline: {e}"

In [ ]:
# Cell 3: Baseline B - Zero-Shot Rewrite
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def run_baseline_zero_shot(assistant, original_query):
    """
    Runs the full RAG pipeline using a simple zero-shot rewritten query.
    """
    # 1. Define the simple zero-shot prompt
    zero_shot_prompt = PromptTemplate.from_template(
        "You are an AI assistant. Rewrite the following query to be more effective for a "
        "semantic search over academic papers. Provide only the single improved query. "
        "Original Query: {question}\n"
        "Rewritten Query:"
    )
    
    # 2. Create and run the rewriting chain
    chain = zero_shot_prompt | assistant.llm | StrOutputParser()
    rewritten_query = chain.invoke({"question": original_query}).strip().strip('"')
    
    # 3. Run the RAG pipeline with the new query
    final_answer = run_rag_pipeline(assistant, rewritten_query, original_query)
    
    return rewritten_query, final_answer

In [ ]:
# Cell 5: Main Execution Loop
# --- Configuration ---
INPUT_CSV_PATH = 'QueryStatistics.csv'  # Your original CSV with all queries
OUTPUT_CSV_PATH = 'baselines_b_and_c_data.csv'
SAMPLE_SIZE = 100 # How many queries to process

# --- Load and Sample Queries ---
df_full = pd.read_csv(INPUT_CSV_PATH)
unique_queries = df_full['Original Query'].unique().tolist()
queries_to_process = unique_queries[:SAMPLE_SIZE]

# --- Process Queries ---
results = []
for query in tqdm(queries_to_process, desc="Generating Baseline Answers"):
    # Run Baseline B
    rewritten_b, answer_b = run_baseline_zero_shot(assistant, query)
    
    # Run Baseline C
    rewritten_c, answer_c = run_baseline_single_pass_best(assistant, query)
    
    results.append({
        'original_query': query,
        'rewritten_query_B_zero_shot': rewritten_b,
        'answer_B_zero_shot': answer_b,
        'rewritten_query_C_single_pass': rewritten_c,
        'answer_C_single_pass': answer_c,
    })

# --- Save Results ---
df_output = pd.DataFrame(results)
df_output.to_csv(OUTPUT_CSV_PATH, index=False)

print(f"\n✅ Done! Baseline data saved to '{OUTPUT_CSV_PATH}'.")

### Complete code

In [3]:
#### import pandas as pd
import re
from tqdm.notebook import tqdm # Use tqdm.notebook for a nice progress bar in Jupyter
import json
from typing import Dict, List, Tuple, Optional, Any, Literal
from dataclasses import dataclass, field
from sentence_transformers import SentenceTransformer, util
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, BaseOutputParser
from langchain.schema import Document, HumanMessage, AIMessage
from langchain.vectorstores import VectorStore
from langchain.chains import LLMChain
# from langchain.llms.base import BaseLLM
from langchain.schema.runnable import Runnable
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
import logging
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import BaseOutputParser, StrOutputParser
from langchain_ollama import OllamaLLM
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer, util
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import ArxivLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import requests
import os
import uuid
import time

# --- IMPORTANT ---
# Make sure your script with the ResearchAssistant class is saved as a .py file
# For example, save it as 'sagerag.py' in the same directory.
from sagerag import ResearchAssistant
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


print("---Starting Evaluation Data Generation ---")

# ==============================================================================
# 1. SETUP: Load models, database, and the assistant
# ==============================================================================
print("1. Initializing models and database...")

# --- Load your NEW, large ChromaDB ---
DB_DIR_NEW = "../Database/Vector-DB-New" 
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2", 
    model_kwargs={'device': 'mps'}
)
vector_db = Chroma(persist_directory=DB_DIR_NEW, embedding_function=embedding_model)

# --- Initialize the LLM ---
llm = OllamaLLM(model="llama3.2", temperature=0.2)

# --- Initialize the main assistant class which we will use as a toolkit ---
assistant = ResearchAssistant(llm=llm, vector_db=vector_db)

print(" Setup Complete. Assistant is ready.")

# ==============================================================================
# 2. HELPER FUNCTIONS: Define the logic for each system
# ==============================================================================
print("\n2. Defining baseline system functions...")

def run_baseline_rag_pipeline(assistant, query_for_retrieval, original_question):
    """A non-interactive function to run the retrieve-and-generate pipeline."""
    try:
        retrieved_docs, _ = assistant.retrieve_documents(queries=[query_for_retrieval], k=5)
        if not retrieved_docs:
            return "Error: Could not retrieve any relevant documents."
        final_answer = assistant.generate_standard_answer(
            question=original_question,
            docs=retrieved_docs,
            max_docs=5
        )
        return final_answer
    except Exception as e:
        return f"Error in RAG pipeline: {e}"
def run_rag_C_pipeline(assistant, query_for_retrieval, original_question):
    """The MID-TIER RAG pipeline for System C. Uses the CITED prompt."""
    try:
        # Retrieve documents using the query directly
        retrieved_docs, _ = assistant.retrieve_documents(queries=[query_for_retrieval], k=5)
        if not retrieved_docs:
            return "Error: Could not retrieve any relevant documents."
        
        # Call the mid-tier generator with citations
        return assistant.generate_cited_answer(original_question, retrieved_docs)
    except Exception as e:
        return f"Error in RAG pipeline: {e}"
# def run_BD_rag_pipeline(assistant, query_for_retrieval, original_question):
#     """A non-interactive function to run the retrieve-and-generate pipeline."""
#     try:
#         retrieved_docs, _ = assistant.retrieve_documents(queries=[query_for_retrieval], k=5)
#         if not retrieved_docs:
#             return "Error: Could not retrieve any relevant documents."
#         final_answer = assistant.generate_premium_answer(
#             question=original_question,
#             docs=retrieved_docs,
#             max_docs=5
#         )
#         return final_answer
#     except Exception as e:
#         return f"Error in RAG pipeline: {e}"

def get_answer_A_original(assistant, original_query):
    """System A: Just use the original query."""
    return run_baseline_rag_pipeline(assistant, original_query, original_query)

def get_answer_B_zero_shot(assistant, original_query):
    """System B: Use a simple zero-shot rewrite."""
    zero_shot_prompt = PromptTemplate.from_template(
        "You are an AI assistant. Rewrite the following query to be more effective for a "
        "semantic search over academic papers. Provide only the single improved query. "
        "Original Query: {question}\nRewritten Query:"
    )
    chain = zero_shot_prompt | assistant.llm | StrOutputParser()
    rewritten_query = chain.invoke({"question": original_query}).strip().strip('"')
    return run_baseline_rag_pipeline(assistant, rewritten_query, original_query)

def get_answer_C_single_pass_best(assistant, original_query):
    """System C: Generate 5 rewrites and auto-select the one with the highest similarity score."""
    
    all_candidates = df_rewrites[df_rewrites['Original Query'] == original_query]['Refined Query'].tolist()
    candidates = [c for c in all_candidates if c != original_query]
    if not candidates:
        return "Error: Could not find pre-generated rewrites for this query."

    scored_candidates = assistant.score_queries(original_query, candidates)
    best_query = scored_candidates[0][0] # The list is sorted, so the first one is the best

    print(f"Auto-Selected Best Rewrite (Score: {scored_candidates[0][1]:.2f}): {best_query}")
    return run_rag_C_pipeline(assistant, best_query, original_query)

# def get_answer_D_sagerag_expert(assistant, original_query, df_all_rewrites):
#     """System D: Simulate an expert user by picking the best of the 5 pre-generated rewrites."""
#     print(f"--- Running System D (Interactive) for Query: [bold cyan]{original_query}[/bold cyan] ---")

#     all_candidates = df_rewrites[df_rewrites['Original Query'] == original_query]['Refined Query'].tolist()
#     candidates = [c for c in all_candidates if c != original_query] # ✅ This new line filters out the original
#     # Display the options to the user (you)
#     query_options = assistant.present_query_options(original_query, candidates)

#     # Get your input
#     selected_indices_input = input("Enter the numbers of the queries to use (e.g., '1,3') > ")
#     try:
#         selected_indices = [int(idx.strip()) for idx in selected_indices_input.split(",")]
#     except ValueError:
#         print("[bold red]Invalid input. Defaulting to original query (0).[/bold red]")
#         selected_indices = [0]

#     # Process your selection
#     selected_queries = []
#     for idx in selected_indices:
#         if 0 <= idx < len(query_options):
#             full_text = query_options[idx][0]
#             # This logic cleans up the query from the pretty-printed format
#             clean_query = re.sub(r'(Original: |Rewrite \d+: )', '', full_text).strip()
#             clean_query = re.sub(r'.*?\[bold\]Query:\[/bold\]\s*', '', clean_query).replace('\n', ' ')
#             selected_queries.append(clean_query)

#     if not selected_queries:
#         print("[bold red]No valid queries selected. Defaulting to original.[/bold red]")
#         selected_queries = [original_query]

#     # Run the appropriate RAG pipeline based on selection count
#     print(f"Retrieving with {len(selected_queries)} selected queries...")
#     if len(selected_queries) > 1:
#         docs, query_docs_map = assistant.retrieve_documents(selected_queries, k=3) # Retrieve fewer docs per query if using multiple
#         return assistant.generate_combined_answer(original_query, query_docs_map)
#     else:
#         return run_BD_rag_pipeline(assistant, selected_queries[0], original_query)


# ==============================================================================
# 3. EXECUTION: Run the pipelines and save the results
# ==============================================================================
print("\n3. Starting the main execution loop...")

# --- Configuration ---
INPUT_CSV_PATH = 'QueryStatistics.csv'
OUTPUT_CSV_PATH = 'human_evaluation_data_ABC.csv'
SAMPLE_SIZE = 100 # Adjust this to 100 or 150 as you need

# --- Load and Sample Queries ---
df_full = pd.read_csv(INPUT_CSV_PATH)
unique_queries = df_full['Original Query'].unique().tolist()
queries_to_process = unique_queries[:SAMPLE_SIZE]

# Create a DataFrame of just the rewrites for easy lookup in System D
df_rewrites = df_full[['Original Query', 'Refined Query']].copy()

# --- Process Queries ---
final_results = []
for query in tqdm(queries_to_process, desc="Processing Queries for All Systems"):
    assistant.clear_memory()
    # Get answer for each system
    ans_A = get_answer_A_original(assistant, query)
    ans_B = get_answer_B_zero_shot(assistant, query)
    ans_C = get_answer_C_single_pass_best(assistant, query)
    # ans_D = get_answer_D_sagerag_expert(assistant, query, df_rewrites)
    
    final_results.append({
        'Query': query,
        'System A Ans': ans_A,
        'System B Ans': ans_B,
        'System C Ans': ans_C,
        # 'System D Ans': ans_D
    })

# --- Save Final Results ---
df_final = pd.DataFrame(final_results)
df_final.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8')

print(f"\n--- 🎉 All Done! ---")
print(f"Generated answers for {len(queries_to_process)} queries.")
print(f"Data for human evaluation saved to: '{OUTPUT_CSV_PATH}'")

--- 🚀 Starting Evaluation Data Generation ---
1. Initializing models and database...


2025-10-27 17:52:08,604 - SageRAG - INFO - Initializing default embedding model
2025-10-27 17:52:13,796 - SageRAG - INFO - Research Assistant initialized successfully


✅ Setup Complete. Assistant is ready.

2. Defining baseline system functions...

3. Starting the main execution loop...


Processing Queries for All Systems:   0%|          | 0/10 [00:00<?, ?it/s]

Conversation memory cleared.

...

Score: 0.3493

╭──────────────────────────────────────── Content Preview ─────────────────────────────────────────╮
│ In Figure 7, we investigate full R-linear convergence of the quasi-error Hk                      │
│ ℓ from (27) as                                                                                   │
│ stated in Theorem 5. The quasi-error is reduced in each ste...                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Source: https://arxiv.org/pdf/2404.07126v1_Iterative_solvers_in_adaptive_FEM

Total unique documents: 5


--- 🎉 All Done! ---
Generated answers for 10 queries.
Data for human evaluation saved to: 'human_evaluation_data_ABC.csv'


# Demo

In [2]:
#### import pandas as pd


import re
from langsmith import traceable

from tqdm.notebook import tqdm # Use tqdm.notebook for a nice progress bar in Jupyter
import json
from typing import Dict, List, Tuple, Optional, Any, Literal
from dataclasses import dataclass, field
from sentence_transformers import SentenceTransformer, util
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, BaseOutputParser
from langchain.schema import Document, HumanMessage, AIMessage
from langchain.vectorstores import VectorStore
from langchain.chains import LLMChain
# from langchain.llms.base import BaseLLM
from langchain.schema.runnable import Runnable
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
import logging
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import BaseOutputParser, StrOutputParser
from langchain_ollama import OllamaLLM
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer, util
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import ArxivLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import requests
import os
import uuid
import time

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- IMPORTANT: Import your main class ---
try:
    from tracing import ResearchAssistant
except ImportError:
    print("="*80)
    print("Error: Could not find 'sagerag.py'.")
    print("Please make sure your 'sagerag.py' file is in the same directory.")
    print("="*80)
    exit()

# --- Setup Console ---
console = Console()

# ==============================================================================
# 1. SETUP: Load models, database, and the assistant
# ==============================================================================
console.print("[bold cyan]Initializing models and database...[/bold cyan]")

try:
    # --- Load your NEW, large ChromaDB ---
    DB_DIR_NEW = "../Database/Vector-DB-New" # Make sure this path is correct
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2", 
        model_kwargs={'device': 'mps'} # 'mps' for Mac. Change to 'cuda' on your server
    )
    vector_db = Chroma(persist_directory=DB_DIR_NEW, embedding_function=embedding_model)

    # --- Initialize the LLM ---
    llm = OllamaLLM(model="llama3.2", temperature=0.2)

    # --- Initialize the main assistant class ---
    assistant = ResearchAssistant(llm=llm, vector_db=vector_db)

    console.print("[bold green]✅ Setup Complete. Assistant is ready.[/bold green]")

except Exception as e:
    console.print(f"[bold red]❌ Error during setup:[/bold red]\n{e}")
    console.print(f"Please check your database path: '{DB_DIR_NEW}'")
    exit()




BATCH_QUERIES = [
    "gpu vs cpu what to use for training?",
    "What attention mechanism do?",
    "What dropout layer for?",
    "What attention mechanism do?",
    "How much data needed for training models for image classification?",
    "What is backprop exactly?",
    "Regularization prevent what?",
    "word embedding vs sentence embedding?",
    "What is transfer learning nlp?",
    "named entity recognition hard why?",
    "language model perplexity mean?",
    "What is few shot learning?"
]

# ==============================================================================
# 2. HELPER FUNCTIONS: Define the logic for each system
# ==============================================================================

def run_baseline_rag_pipeline(assistant, query_for_retrieval, original_question):
    """A non-interactive function to run the retrieve-and-generate pipeline."""
    try:
        retrieved_docs, _ = assistant.retrieve_documents(queries=[query_for_retrieval], k=3)
        if not retrieved_docs:
            return "Error: Could not retrieve any relevant documents."
        final_answer = assistant.generate_standard_answer(
            question=original_question,
            docs=retrieved_docs,
            max_docs=3
        )
        return final_answer
    except Exception as e:
        return f"Error in RAG pipeline: {e}"

def get_answer_A_original(assistant, original_query):
    """System A: Just use the original query."""
    console.print("[bold]Running System A (Baseline)...[/bold]")
    return run_baseline_rag_pipeline(assistant, original_query, original_query)

def get_answer_B_zero_shot(assistant, original_query):
    """System B: Use a simple zero-shot rewrite."""
    console.print("[bold]Running System B (Zero-Shot Rewrite)...[/bold]")
    zero_shot_prompt = PromptTemplate.from_template(
        "You are an AI assistant. Rewrite the following query to be more effective for a "
        "semantic search over academic papers. Provide only the single improved query. "
        "Original Query: {question}\nRewritten Query:"
    )
    chain = zero_shot_prompt | assistant.llm | StrOutputParser()
    rewritten_query = chain.invoke({"question": original_query}).strip().strip('"')
    
    console.print(f"[dim]System B used rewrite: '{rewritten_query}'[/dim]")
    return run_baseline_rag_pipeline(assistant, rewritten_query, original_query)

def get_answer_C_dynamic_best(assistant, original_query):
    """
    System C (Dynamic): Generate 5 rewrites and auto-select the best one.
    This is the correct interactive version, it does not use a CSV.
    """
    console.print("[bold]Running System C (Auto-Best of 5)...[/bold]")
    
    # 1. Generate new rewrites on the fly
    candidates = assistant.suggest_rewrites(original_query)
    if not candidates:
        console.print("[yellow]System C could not generate rewrites. Using original query.[/yellow]")
        return run_baseline_rag_pipeline(assistant, original_query, original_query)
        
    # 2. Score the new rewrites
    scored_candidates = assistant.score_queries(original_query, candidates)
    
    # 3. Select the best one
    best_query = scored_candidates[0][0]
    best_score = scored_candidates[0][1]

    console.print(f"[dim]System C auto-selected rewrite (Score: {best_score:.2f}): '{best_query}'[/dim]")
    
    # 4. Generate the answer using the cited pipeline
    # We use 'generate_cited_answer' for System C as per your original script
    try:
        retrieved_docs, _ = assistant.retrieve_documents(queries=[best_query], k=5)
        if not retrieved_docs:
            return "Error: Could not retrieve any relevant documents for System C."
        return assistant.generate_cited_answer(original_query, retrieved_docs)
    except Exception as e:
        return f"Error in RAG C pipeline: {e}"

# ==============================================================================
# 3. MAIN DEMO LOOP
# ==============================================================================

def main_demo():
    console.print(Panel.fit(
        Markdown("# SageRAG Baseline Comparison Demo\n\nThis script will run Systems A, B, and C for any query you provide.\nTo run the full System D (Human-in-the-Loop) demo, run `app.py`."),
        title="Welcome",
        border_style="bold green"
    ))
    
    for query in BATCH_QUERIES:
        assistant.clear_memory()

        # # --- Step 1: Rate Query ---
        # console.print(Panel("Step 1: Rating Query...", style="bold cyan", padding=1))
        # rating = assistant.rate_query(query)
        # console.print(f"[bold]Query Rating:[/bold] {rating['rating']}/5")
        # console.print(f"[bold]Explanation:[/bold] {rating['explanation']}")
        
        # --- Step 2: System A ---
        # console.print(Panel("Step 2: Generating System A (Baseline) Answer...", style="bold cyan", padding=1))
        ans_A = get_answer_A_original(assistant, query)
        console.print(Panel(Markdown(ans_A), title="System A (Baseline)", border_style="red"))

        # # --- Step 3: System B ---
        # console.print(Panel("Step 3: Generating System B (Zero-Shot) Answer...", style="bold cyan", padding=1))
        # ans_B = get_answer_B_zero_shot(assistant, query)
        # console.print(Panel(Markdown(ans_B), title="System B (Zero-Shot)", border_style="yellow"))

        # # --- Step 4: System C ---
        # console.print(Panel("Step 4: Generating System C (Auto-Best) Answer...", style="bold cyan", padding=1))
        # ans_C = get_answer_C_dynamic_best(assistant, query)
        # console.print(Panel(Markdown(ans_C), title="System C (Auto-Best)", border_style="green"))

    console.print("[bold green]Baseline demo complete. Goodbye![/bold green]")

if __name__ == "__main__":
    main_demo()

Initializing models and database...

/var/folders/5n/8m4t00412h97v592vh5y0pgc0000gn/T/ipykernel_33986/3518748191.py:65: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/var/folders/5n/8m4t00412h97v592vh5y0pgc0000gn/T/ipykernel_33986/3518748191.py:69: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=DB_DIR_NEW, embedding_function=

✅ Setup Complete. Assistant is ready.

╭──────────────────────────────────────────────────── Welcome ────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃                                      SageRAG Baseline Comparison Demo                                       ┃ │
│ ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛ │
│                                                                                                                 │
│ This script will run Systems A, B, and C for any query you provide. To run the full System D                    │
│ (Human-in-the-Loop) demo, run app.py.                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Conversation memory cleared.

...

╭──────────────────────────────────────── Content Preview ─────────────────────────────────────────╮
│ [88] J. Ma, H. Xie, G. Han, S.-F. Chang, A. Galstyan, and W. Abd-                                │
│ Almageed, “Partner-assisted learning for few-shot image classiﬁ-                                 │
│ cation,” in Proceed...                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Source: https://arxiv.org/pdf/2202.07568v6_StratDef:_Strategic_Defense_Against_Adversarial_At

Total unique documents: 3

╭────────────────────────────────────────────── System A (Baseline) ──────────────────────────────────────────────╮
│ Few-shot learning is a special case of learning with small datasets, where one is provided either with just     │
│ semantic information of the target classes (zero-shot learning), only one labelled example per class (1-shot    │
│ learning) or just few samples (general few-shot learning).                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Baseline demo complete. Goodbye!

In [3]:
#### File: 2_collect_systemD_choices.py
import pandas as pd
import os
from tqdm import tqdm
from sagerag import ResearchAssistant, OllamaLLM, Chroma, HuggingFaceEmbeddings
from rich.console import Console

# Rich console for better output
console = Console()

print("--- 🚀 Starting Interactive Choice Collection for System D (Resumable) ---")

# --- 1. SETUP ---
print("1. Initializing...")

# --- Configuration ---
DB_PATH = "Database/Vector-DB-New"
INPUT_CSV_PATH = 'QueryStatistics.csv'
CHOICES_OUTPUT_PATH = 'system_D_choices.csv'

# --- KEY CHANGE: Define the range of queries to process ---
START_INDEX = 10  # Start from the 11th query (index 10)
TOTAL_QUERIES = 200 # Process up to the 200th query

# --- KEY CHANGE: Set device to 'cuda' for the server ---
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2", 
    model_kwargs={'device': 'mps'}
)
vector_db = Chroma(persist_directory=DB_PATH, embedding_function=embedding_model)
llm = OllamaLLM(model="llama3.2", temperature=0.2)
assistant = ResearchAssistant(llm=llm, vector_db=vector_db)
print("✅ Setup Complete.")


# --- 2. EXECUTION ---

# --- Load all data ---
df_full = pd.read_csv(INPUT_CSV_PATH)
df_full.columns = df_full.columns.str.strip() # Clean column names
all_unique_queries = df_full['Original Query'].unique().tolist()
df_rewrites = df_full[['Original Query', 'Refined Query']].copy()

# --- KEY CHANGE: Determine which queries still need to be processed ---
queries_to_process = all_unique_queries[START_INDEX:TOTAL_QUERIES]

# --- Resumability Logic ---
completed_queries = set()
if os.path.exists(CHOICES_OUTPUT_PATH):
    try:
        df_existing = pd.read_csv(CHOICES_OUTPUT_PATH)
        completed_queries = set(df_existing['Query'].tolist())
        print(f"Found {len(completed_queries)} previously completed queries. Resuming...")
        queries_to_process = [q for q in queries_to_process if q not in completed_queries]
    except pd.errors.EmptyDataError:
        print("Output file is empty. Starting fresh for the specified range.")

if not queries_to_process:
    print("\n--- ✅ All queries in the specified range are already complete! ---")
else:
    print(f"\n--- Starting Choice Selection for {len(queries_to_process)} remaining queries ---")
    
    # Use tqdm for a progress bar
    for query in tqdm(queries_to_process, desc="Collecting Choices"):
        console.print("\n" + "="*80, style="bold blue")
        console.print(f"Query: [bold cyan]{query}[/bold cyan]")
        console.print("="*80, style="bold blue")

        all_candidates = df_rewrites[df_rewrites['Original Query'] == query]['Refined Query'].tolist()
        candidates = [c for c in all_candidates if c != query] # Filter out original

        # Display options
        query_options = assistant.present_query_options(query, candidates)

        # Get user input
        selected_indices_input = input("Enter the numbers of the queries to use (e.g., '0,2,3') > ")

        # Prepare a new choice to be appended
        new_choice = {
            'Query': query,
            'Selected Indices': selected_indices_input # Store the raw input
        }

        # --- KEY CHANGE: Append result immediately ---
        df_new_choice = pd.DataFrame([new_choice])
        file_exists = os.path.isfile(CHOICES_OUTPUT_PATH)
        df_new_choice.to_csv(
            CHOICES_OUTPUT_PATH,
            mode='a',
            header=not file_exists or os.path.getsize(CHOICES_OUTPUT_PATH) == 0,
            index=False,
            encoding='utf-8'
        )

    print(f"\n--- ✅ Choice Collection Complete! ---")
    print(f"Your selections have been saved to: '{CHOICES_OUTPUT_PATH}'")

--- 🚀 Starting Interactive Choice Collection for System D (Resumable) ---
1. Initializing...


2025-10-28 03:16:02,852 - SageRAG - INFO - Initializing default embedding model
2025-10-28 03:16:07,085 - SageRAG - INFO - Research Assistant initialized successfully


✅ Setup Complete.
Found 13 previously completed queries. Resuming...

--- Starting Choice Selection for 186 remaining queries ---


================================================================================

Query: Neural network depth vs width?

...

Confidence: 0.72

5. Rewrite 5: "How do different data imputation techniques (mean/median/mode, regression-based/iterative 
imputation) handle varying levels of missingness and dataset characteristics? What are the advantages/disadvantages
of each approach?"

Confidence: 0.67

Enter the numbers of the queries to use (e.g., '0,2,3') >  2



--- ✅ Choice Collection Complete! ---
Your selections have been saved to: 'system_D_choices.csv'


In [1]:
# File: 3_generate_systemD_and_merge.py
import pandas as pd
import re
from tqdm import tqdm
from sagerag import ResearchAssistant, OllamaLLM, Chroma, HuggingFaceEmbeddings # Assuming your class is in sagerag.py
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


print("--- 🚀 Starting System D Generation and Final Merge ---")

# --- 1. SETUP ---
print("1. Initializing models and database...")
# (Initialize llm, embeddings, vector_db, assistant - same as script 1 & 2)
# --- Load your NEW, large ChromaDB ---
DB_DIR_NEW = "../Database/Vector-DB-New"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
vector_db = Chroma(persist_directory=DB_DIR_NEW, embedding_function=embedding_model)
llm = OllamaLLM(model="llama3.2", temperature=0.2)
assistant = ResearchAssistant(llm=llm, vector_db=vector_db)
print("✅ Setup Complete.")

# --- 2. HELPER FUNCTIONS ---
# (Need generate_hypothetical_answer and the premium RAG generation functions)
def generate_hypothetical_answer(assistant, query):
    """Generates a hypothetical answer to be used for retrieval (HyDE)."""
    hyde_prompt = PromptTemplate.from_template(
        "Generate a detailed, high-quality paragraph that is a hypothetical "
        "but plausible answer to the following research question. Do not say it is hypothetical. "
        "Just generate the answer directly.\n\nQuestion: {question}\n\nAnswer:"
    )
    chain = hyde_prompt | assistant.llm | StrOutputParser()
    return chain.invoke({"question": query})

def run_system_D_pipeline(assistant, original_query, selected_indices_str, df_rewrites):
    """
    Runs the System D pipeline using pre-selected indices, and now correctly calls
    the right generation function based on the number of selected queries.
    """
    # --- Part 1: Determine which queries were selected (same as before) ---
    all_candidates = df_rewrites[df_rewrites['Original Query'] == original_query]['Refined Query'].tolist()
    candidates = [c for c in all_candidates if c != original_query]
    
    query_options = [("Original: " + original_query, 1.0)]
    scored_rewrites = assistant.score_queries(original_query, candidates)
    for i, (query_text, score) in enumerate(scored_rewrites, 1):
         query_options.append((f"Rewrite {i}: {query_text}", score))

    try:
        selected_indices = [int(idx.strip()) for idx in selected_indices_str.split(",")]
    except (ValueError, AttributeError):
        selected_indices = [0]

    selected_queries = []
    for idx in selected_indices:
        if 0 <= idx < len(query_options):
            full_text = query_options[idx][0]
            clean_query = re.sub(r'(Original: |Rewrite \d+: )', '', full_text).strip()
            selected_queries.append(clean_query)
            
    if not selected_queries: 
        selected_queries = [original_query]

    # --- 💡 Part 2: The CRITICAL Logic Change 💡 ---
    # Use HyDE for retrieval based on the first selected query
    hyde_document = generate_hypothetical_answer(assistant, selected_queries[0])
    
    # This block now acts as the "traffic cop"
    if len(selected_queries) > 1:
        # --- Case 1: MULTIPLE queries were selected ---
        print(f"-> Using multi-query pipeline for {len(selected_queries)} queries...")
        # Retrieve documents for each selected query, which returns a dictionary
        _, query_docs_map = assistant.retrieve_documents(queries=selected_queries, k=3)
        # Call the function designed for dictionaries
        return assistant.generate_combined_answer(original_query, query_docs_map)
    else:
        # --- Case 2: A SINGLE query was selected ---
        print(f"-> Using single-query pipeline...")
        # Retrieve documents for the single query, which returns a list
        retrieved_docs, _ = assistant.retrieve_documents(queries=[hyde_document], k=5)
        # Call the function designed for lists
        return assistant.generate_premium_answer(original_query, retrieved_docs)

# --- 3. EXECUTION ---
CHOICES_INPUT_PATH = 'system_D_choices.csv'
BASELINES_INPUT_PATH = 'human_evaluation_data_ABC.csv'
FINAL_OUTPUT_PATH = 'human_evaluation_data_final.csv'
INPUT_CSV_PATH = 'QueryStatistics.csv' # Needed for df_rewrites lookup

print(f"\n3. Loading choices from '{CHOICES_INPUT_PATH}'...")
df_choices = pd.read_csv(CHOICES_INPUT_PATH)

print(f"Loading original rewrite data from '{INPUT_CSV_PATH}'...")
df_full = pd.read_csv(INPUT_CSV_PATH)
df_full.columns = df_full.columns.str.strip()
df_rewrites = df_full[['Original Query', 'Refined Query']].copy()


system_d_results = []
for index, row in tqdm(df_choices.iterrows(), total=df_choices.shape[0], desc="Generating System D Answers"):
    query = row['Query']
    choices_str = row['Selected Indices']
    assistant.clear_memory() # Clear memory for each query
    
    ans_D = run_system_D_pipeline(assistant, query, choices_str, df_rewrites)
    system_d_results.append({
        'Query': query,
        'System D Ans': ans_D
    })

df_system_d = pd.DataFrame(system_d_results)

# --- 4. MERGE RESULTS ---
print(f"\n4. Merging results...")
df_baselines = pd.read_csv(BASELINES_INPUT_PATH)
df_final = pd.merge(df_baselines, df_system_d, on='Query')

# Save the final combined file
df_final.to_csv(FINAL_OUTPUT_PATH, index=False, encoding='utf-8')

print(f"\n--- ✅ All Done! ---")
print(f"Final merged data saved to: '{FINAL_OUTPUT_PATH}'")

--- 🚀 Starting System D Generation and Final Merge ---
1. Initializing models and database...


/var/folders/5n/8m4t00412h97v592vh5y0pgc0000gn/T/ipykernel_19214/1177039783.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
/var/folders/5n/8m4t00412h97v592vh5y0pgc0000gn/T/ipykernel_19214/1177039783.py:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma imp

✅ Setup Complete.

3. Loading choices from 'system_D_choices.csv'...
Loading original rewrite data from 'QueryStatistics.csv'...


Generating System D Answers:   0%|                       | 0/13 [00:00<?, ?it/s]

Conversation memory cleared.

Generating System D Answers:   0%|                       | 0/13 [00:00<?, ?it/s]


ConnectError: [Errno 61] Connection refused

In [4]:
# In your script's EXECUTION section

# --- Load and Sample Queries ---
INPUT_CSV_PATH = 'QueryStatistics.csv'
df_full = pd.read_csv(INPUT_CSV_PATH)

# ✅ ADD THIS LINE TO SEE THE REAL COLUMN NAMES
print("Columns in CSV:", df_full.columns) 

unique_queries = df_full['Original Query'].unique().tolist()
# ... rest of your code

Columns in CSV: Index(['Query ID', 'Original Query', 'Query Type', 'Refined Query',
       'LLM Answer', 'Query Score', 'Unnamed: 6'],
      dtype='object')


In [8]:
import pickle
from tqdm.notebook import tqdm
from sagerag import ResearchAssistant
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def generate_question_from_chunk(assistant, chunk_text):
    """Uses an LLM to generate a question that the chunk_text can answer."""
    
    question_generation_prompt = PromptTemplate.from_template(
        "You are an expert researcher. Given the following paragraph from a scientific paper, "
        "generate one clear, specific question that this text could perfectly answer. "
        "Do not ask a question that is too broad. Focus only on the information present in the text. "
        "Provide only the question and nothing else.\n\n"
        "Text: \"{context}\"\n\n"
        "Question:"
    )
    
    chain = question_generation_prompt | assistant.llm | StrOutputParser()
    question = chain.invoke({"context": chunk_text}).strip()
    return question

# --- Example Usage in your notebook ---

# Load the chunks you created earlier
with open("../Database/processed_chunks.pkl", "rb") as f:
    all_chunks = pickle.load(f)

# Create a sample of 200 chunks
import random
chunk_sample = random.sample(all_chunks, 200)

golden_dataset = []
for chunk in tqdm(chunk_sample, desc="Generating Golden Questions"):
    question = generate_question_from_chunk(assistant, chunk.page_content)
    golden_dataset.append({
        "generated_question": question,
        "source_chunk_text": chunk.page_content,
        "source_metadata": chunk.metadata
    })

# Save your new evaluation dataset
df_golden = pd.DataFrame(golden_dataset)
df_golden.to_csv("golden_evaluation_questions.csv", index=False)

Generating Golden Questions:   0%|          | 0/200 [00:00<?, ?it/s]

In [5]:
!pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python
